In [1]:
# Import Libraries
import pandas as pd
import folium
import random
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display

/Users/jonathan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
df_updated = pd.read_csv('df_updated.csv')

In [ ]:
# Initialize the map, set to a central point in Maine (or adjust as needed)
map_center = [44.6939, -69.3815]  # Central Maine coordinates (latitude, longitude)
m = folium.Map(location=map_center, zoom_start=7)

# Group data by water body, town, and county
grouped = df_updated.groupby(['WATER', 'TOWN', 'COUNTY'])

# Function to update the map based on selected species, date filters (Spring/Fall), and search
def update_map(selected_species, show_spring, show_fall, search_term, min_qty):
    # Clear the map before adding new markers
    m = folium.Map(location=map_center, zoom_start=7)

    # Filter the grouped data based on the search term
    filtered_groups = []
    for (water_name, town_name, county_name), group in grouped:
        # Check if the search term is in any of the fields (water, town, or county)
        if (search_term.lower() in water_name.lower() or
            search_term.lower() in town_name.lower() or
            search_term.lower() in county_name.lower()):
            filtered_groups.append((water_name, town_name, county_name, group))

    # Add markers to the map for each unique water body, town, and county combination
    for (water_name, town_name, county_name, group) in filtered_groups:
        species_list = []
        qty_list = []
        size_list = []
        date_list = []

        # Loop through each row in the group and collect the data
        for _, row in group.iterrows():
            species_list.append(row['SPECIES'])
            qty_list.append(row['QTY'])
            size_list.append(row['SIZE (inch)'])
            date_list.append(row['DATE'])

        # Filter data by species and date (Spring/Fall)
        filtered_rows = []
        for species, qty, size, date in zip(species_list, qty_list, size_list, date_list):
            month = pd.to_datetime(date).month  # Get the month from the date
            if (show_spring and month <= 6) or (show_fall and month >= 7):
                if species in selected_species and qty >= min_qty:  # Filter by min_qty
                    filtered_rows.append((species, qty, size, date))

        # Only display markers if there is data to show after filtering
        if filtered_rows:
            popup_text = f"""
            <b>Water Body:</b> {water_name}<br>
            <b>Town:</b> {town_name}<br>
            <b>County:</b> {county_name}<br>
            <b>Stocking Data:</b><br>
            <ul>
            """
            for species, qty, size, date in filtered_rows:
                popup_text += f"""
                <li><b>{species}</b> - {qty} fish, Size: {size} inches, Date: {date}</li>
                """
            popup_text += "</ul>"

            # Use the average coordinates of the water body/town combination
            avg_x = group['X_coord'].mean()
            avg_y = group['Y_coord'].mean()

            # Offset coordinates slightly if there are multiple towns associated with the same water body
            offset_x = random.uniform(-0.001, 0.001)  # Random offset for longitude
            offset_y = random.uniform(-0.001, 0.001)  # Random offset for latitude

            # Add a single marker for the water body/town combo with grouped popup data
            folium.Marker(
                location=[avg_y + offset_y, avg_x + offset_x],  # Apply offset to average coordinates
                popup=folium.Popup(popup_text, max_width=300),
                icon=folium.Icon(color='green')  # Popup with the grouped stocking data
            ).add_to(m)

    # Display the updated map
    display(m)

# List of unique species in your dataset
species_list = df_updated['SPECIES'].unique()

# Create checkboxes for each species
checkboxes = widgets.SelectMultiple(
    options=species_list,
    value=species_list.tolist(),  # Start with all species selected
    description='Species:',
    disabled=False
)

# Create checkboxes for Spring and Fall Stocking
spring_checkbox = widgets.Checkbox(
    value=False,
    description='Spring Stocking (Jan-Jun)',
    disabled=False
)

fall_checkbox = widgets.Checkbox(
    value=False,
    description='Fall Stocking (Jul-Dec)',
    disabled=False
)

# Create a search bar to filter by water body, town, or county
search_bar = widgets.Text(
    value='',
    description='Search:',
    placeholder='Type water body, town, or county'
)

# Create a text box for users to enter the minimum quantity of fish
qty_textbox = widgets.IntText(
    value=0,  # Default value
    description='Min Qty:',
    min=0,  # Minimum allowed value
    step=1,  # Step size for the input
    style={'description_width': 'initial'}
)

# Display the checkboxes, text box, and search bar, and link them to the map update function
interact(update_map, 
         selected_species=checkboxes, 
         show_spring=spring_checkbox, 
         show_fall=fall_checkbox, 
         search_term=search_bar, 
         min_qty=qty_textbox)

# Save the map to an HTML file (optional)
m.save("FishMap_2024.html")


interactive(children=(SelectMultiple(description='Species:', index=(0, 1, 2, 3, 4, 5), options=('BROOK TROUT',…